In [4]:
import requests
import json
import re

TOGETHER_API_KEY = "6b8814433504401555d7271a05a050d2b18e6648cb753fb3bc848d618a003583"

def parse_response(raw_response):
    """Parse the response and extract JSON object with fallback methods."""
    if isinstance(raw_response, dict):
        if 'output' in raw_response and isinstance(raw_response['output'], dict):
            choices = raw_response['output'].get('choices', [])
            if choices and isinstance(choices[0], dict):
                text = choices[0].get('text', '')
                try:
                    return json.loads(text.strip())
                except json.JSONDecodeError:
                    pass
        choices = raw_response.get('choices', [])
        if choices and isinstance(choices[0], dict):
            text = choices[0].get('text', '')
            try:
                return json.loads(text.strip())
            except json.JSONDecodeError:
                pass

    text = str(raw_response)
    try:
        text = re.sub(r'```json|```', '', text)
        return json.loads(text.strip())
    except json.JSONDecodeError:
        score = 0
        reasons = []
        score_match = re.search(r'"scam_score":\s*(\d+)', text)
        if score_match:
            score = int(score_match.group(1))
        reasons_match = re.search(r'"reasons":\s*\[([^\]]*)\]', text)
        if reasons_match:
            reasons = [r.strip(' "\'') for r in reasons_match.group(1).split(',') if r.strip()]
        return {
            "scam_score": max(0, min(100, score)),
            "reasons": reasons if reasons else ["No specific reasons provided"]
        }

def tag_email(input_email):
    """Analyze an email for scam indicators based on structured input."""
    prompt = f"""Analyze this email for scam indicators. Return ONLY JSON with:
- "scam_score" (0-100)
- "reasons" (array of strings)

Email Details:
From: {input_email.get('from')}
To: {', '.join(input_email.get('to', []))}
CC: {', '.join(input_email.get('cc', []))}
Subject: {input_email.get('subject')}
Body: {input_email.get('message')}
Headers: {input_email.get('headers')}

Example Output:
{{
  "scam_score": 85,
  "reasons": ["suspicious link", "urgent language"]
}}"""

    try:
        headers = {
            "Authorization": f"Bearer {TOGETHER_API_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "prompt": prompt,
            "max_tokens": 300,
            "temperature": 0.1,
            "stop": ["}\n"],
            "response_format": {"type": "json_object"}
        }

        response = requests.post(
            "https://api.together.xyz/inference",
            headers=headers,
            json=data,
            timeout=10
        )
        response.raise_for_status()

        result = parse_response(response.json())
        return {
            "scam_score": result.get("scam_score", 0),
            "reasons": result.get("reasons", ["Could not parse response"])
        }

    except Exception as e:
        return {
            "scam_score": 0,
            "reasons": [f"API Error: {str(e)}"]
        }

# Example test cases
test_cases = [
    {
        "from": "support@paypal.com",
        "to": ["user@example.com"],
        "cc": [],
        "subject": "Account Suspension Notice",
        "message": "Click now: http://fake-paypal.com to verify your account",
        "headers": ["X-Mailer: PhishMailer", "X-Originating-IP: 192.168.1.1"]
    },

]

print("Running test cases:\n")
for i, email in enumerate(test_cases, 1):
    print(f"Test Case {i}: {email['subject']}")
    result = tag_email(email)
    print(json.dumps(result, indent=2))
    print("-" * 50)


Running test cases:

Test Case 1: Account Suspension Notice
{
  "scam_score": 85,
  "reasons": [
    "suspicious link",
    "urgent language"
  ]
}
--------------------------------------------------
